In [2]:
from symmer import PauliwordOp, QuantumState
import qiskit
import numpy as np
from basis_states import *
from hamiltonian import *

In [3]:
N = 6

In [4]:
ham = H(N)

In [9]:
mesons, baryons = get_basis(N)

In [10]:
print("To map to the meson sector, we decrease from ", 2*N, "qubits to ", int(np.ceil(np.log2(len(mesons)))))
print("To map to the baryon sector, we decrease from ", 2*N, "qubits to ", int(np.ceil(np.log2(len(baryons)))))

To map to the meson sector, we decrease from  12 qubits to  9
To map to the baryon sector, we decrease from  12 qubits to  8


In [11]:
def one_hot_to_binary_encoding(states):
    new_nqubits = int(np.ceil(np.log2(len(states))))
    return {format(i, '0' + str(new_nqubits) + 'b'):states[i] for i in range(len(states))}

In [1]:
def H_sector(hamiltonian, states):
    size = len(states)
    H_sector_matrix = np.zeros((size, size), dtype = complex)
    
    for i in range(len(states)):
        for j in range(len(states)):
            H_sector_matrix[i][j] = states[i].dagger * hamiltonian * states[j]
    return PauliwordOp.from_matrix(H_sector_matrix)

In [13]:
oh_binary_dict = one_hot_to_binary_encoding(mesons)

In [14]:
oh_binary_dict

{'000000000':  1.000+0.000j |000000111111>,
 '000000001':  1.000+0.000j |000001101111>,
 '000000010':  1.000+0.000j |000001111011>,
 '000000011':  1.000+0.000j |000001111110>,
 '000000100':  1.000+0.000j |000010011111>,
 '000000101':  1.000+0.000j |000010110111>,
 '000000110':  1.000+0.000j |000010111101>,
 '000000111':  1.000+0.000j |000011001111>,
 '000001000':  1.000+0.000j |000011011011>,
 '000001001':  1.000+0.000j |000011011110>,
 '000001010':  1.000+0.000j |000011100111>,
 '000001011':  1.000+0.000j |000011101101>,
 '000001100':  1.000+0.000j |000011110011>,
 '000001101':  1.000+0.000j |000011110110>,
 '000001110':  1.000+0.000j |000011111001>,
 '000001111':  1.000+0.000j |000011111100>,
 '000010000':  1.000+0.000j |000100101111>,
 '000010001':  1.000+0.000j |000100111011>,
 '000010010':  1.000+0.000j |000100111110>,
 '000010011':  1.000+0.000j |000101101011>,
 '000010100':  1.000+0.000j |000101101110>,
 '000010101':  1.000+0.000j |000101111010>,
 '000010110':  1.000+0.000j |000

In [15]:
H_meson = H_sector(ham, mesons)

/Users/carterm.gustin/anaconda3/envs/phd/lib/python3.9/site-packages/symmer/operators/base.py:400: ComplexWarning: Casting complex values to real discards the imaginary part
  temp_mat[:matrix.shape[0],


Building operator via projectors diag elements:   0%|          | 0/399 [00:00<?, ?it/s]

Building operator via projectors off-diag elements:   0%|          | 0/3120 [00:00<?, ?it/s]

In [16]:
#meson_sector
vals, vecs = np.linalg.eigh(H_meson.to_sparse_matrix.toarray())

index = 0
ground_state = QuantumState.from_array(vecs[:, index].reshape([-1, 1]))
ground_energy = vals[index]

In [17]:
psi = ground_state.sort('decreasing').cleanup(1e-3)
psi_OH = QuantumState([0]*ham.n_qubits)*0

for state in psi.to_dictionary:
    psi_OH += oh_binary_dict[state] * psi.to_dictionary[state]


In [27]:
vals

array([-160.6103342 , -158.88254922, -158.87022205, -158.36502594,
       -158.16719261, -158.09747163, -158.01746284, -157.90659198,
       -157.36959786, -156.43170279, -156.33562731, -156.26915007,
       -156.20661014, -156.16015385, -156.07012354, -156.01400024,
       -155.9409678 , -155.90556519, -155.71230301, -155.69427399,
       -155.64665834, -155.60779921, -155.60312128, -155.49135704,
       -155.44578788, -155.26623218, -155.2544885 , -153.98055852,
       -153.96980653, -153.65891835, -153.64688523, -153.43078365,
       -153.337385  , -153.25311399, -152.86324816, -151.26874213,
         -3.74276225,   -3.72609056,   -3.24362109,   -3.00267573,
         -2.95531259,   -1.81006745,   -1.56928365,   -1.02480637,
         -0.9065711 ,   -0.87696994,   -0.83231741,   -0.65353855,
         -0.63228196,   -0.55731553,    0.        ,    0.        ,
          0.        ,    0.        ,    0.        ,    0.        ,
          0.        ,    0.        ,    0.        ,    0.     

In [26]:

psi_OH.sort('decreasing')

 0.629+0.000j |100100110011> +
-0.629+0.000j |011000110011> +
-0.136+0.000j |100101100011> +
-0.136+0.000j |011010010011> +
-0.128+0.000j |100100110110> +
-0.128+0.000j |011000111001> +
 0.125+0.000j |011000110110> +
 0.125+0.000j |100100111001> +
 0.125+0.000j |011001100011> +
 0.125+0.000j |100110010011> +
-0.123+0.000j |100100011011> +
-0.123+0.000j |011000100111> +
 0.121+0.000j |011000011011> +
 0.121+0.000j |100100100111> +
 0.031+0.000j |100101100110> +
-0.031+0.000j |011010011001> +
-0.030+0.000j |100101001011> +
-0.030+0.000j |011010000111> +
 0.030+0.000j |100111000011> +
-0.030+0.000j |011011000011> +
-0.029+0.000j |011000111100> +
 0.029+0.000j |100100111100> +
-0.028+0.000j |011001100110> +
 0.028+0.000j |100110011001> +
 0.027+0.000j |011001001011> +
 0.027+0.000j |100110000111> +
-0.026+0.000j |011000001111> +
 0.026+0.000j |100100001111> +
-0.026+0.000j |100101101001> +
 0.026+0.000j |011010010110> +
 0.026+0.000j |100100011110> +
 0.026+0.000j |011000101101> +
-0.025+0

In [22]:
from symmer.utils import exact_gs_energy

gs, psi = exact_gs_energy(ham.to_sparse_matrix)

In [23]:
psi.sort('decreasing')

-0.047-0.627j |100100110011> +
 0.047+0.627j |011000110011> +
 0.010+0.136j |100101100011> +
 0.010+0.136j |011010010011> +
 0.010+0.127j |100100110110> +
 0.010+0.127j |011000111001> +
-0.009-0.125j |011000110110> +
-0.009-0.125j |100100111001> +
-0.009-0.125j |011001100011> +
-0.009-0.125j |100110010011> +
 0.009+0.123j |100100011011> +
 0.009+0.123j |011000100111> +
-0.009-0.120j |011000011011> +
-0.009-0.120j |100100100111> +
-0.002-0.030j |100101100110> +
 0.002+0.030j |011010011001> +
 0.002+0.030j |011010000111> +
 0.002+0.030j |100101001011> +
-0.002-0.030j |100111000011> +
 0.002+0.030j |011011000011> +
 0.002+0.029j |011000111100> +
-0.002-0.029j |100100111100> +
 0.002+0.028j |011001100110> +
-0.002-0.028j |100110011001> +
-0.002-0.027j |100110000111> +
-0.002-0.027j |011001001011> +
 0.002+0.026j |011000001111> +
-0.002-0.026j |100100001111> +
-0.002-0.026j |011010010110> +
 0.002+0.026j |100101101001> +
-0.002-0.026j |100100011110> +
-0.002-0.026j |011000101101> +
 0.002+0

In [26]:
np.abs(psi_OH.dagger * psi)**2

0.9999684364232204

In [31]:
(Qx_tot(N) * psi).cleanup(1e-3)